In [ ]:
# Load table into pandas df

import pandas as pd

ratings_df = pd.read_csv('sample_data/ratings.csv')

print(ratings_df.head())

In [ ]:
# Split dataset into train (70%), dev (15%), test (15%)

from sklearn.model_selection import train_test_split

train_df, temp_df = train_test_split(ratings_df, test_size=0.3, random_state=42)
dev_df, test_df = train_test_split(temp_df, test_size=0.5, random_state=42) 

High-level explanation of SGD: 
1. We initialize a user embedding table and an item embedding table
2. We use DataLoader to set up our data. 
3. Forward pass
     Let's say user of id 1 rated movie of id 4.
     We look up the user embedding of userid=1 and the item embedding of itemid=4.
     We perform element-wise multiplication of the two embeddings and sum the result
     This is the predicted rating
4. Backward pass
     We compare the predicted rating with the label rating using MSE.
     Update the user and item embedding tables accordingly.

In [ ]:
import torch
import numpy as np
from torch.autograd import Variable
from torch.utils.data.dataset import Dataset
from torch.utils.data import DataLoader
import torch.nn as nn
import torch.optim as optim
from sklearn.metrics import mean_absolute_error, mean_squared_error

In [ ]:
# Define the model: we create user and item embedding tables for each unique user and unique item; 
# forward pass by element-wise multiplication and summation

class MatrixFactorization(torch.nn.Module):
    def __init__(self, n_users, n_items, n_factors=20):
        super().__init__()
        # create user embeddings: each user is a tensor of length n_factors.
        self.user_factors = torch.nn.Embedding(n_users, n_factors)
        # create item embeddings: each item is a tensor of length n_factors
        self.item_factors = torch.nn.Embedding(n_items, n_factors)
        self.user_factors.weight.data.uniform_(0, 0.05)
        self.item_factors.weight.data.uniform_(0, 0.05)

    def forward(self, data):
        users, items = data[:,0], data[:,1]
        return (self.user_factors(users) * self.item_factors(items)).sum(1)

    def predict(self, user, item):
        return (self.user_factors(user) * self.item_factors(item)).sum(1)

In [ ]:
# We need this to load the ratings pandas df into torch tensors. 

class Loader(Dataset):
    def __init__(self, ratings_df):
        self.ratings = ratings_df.copy()
        users = ratings_df.userId.unique()
        movies = ratings_df.movieId.unique()

        self.userid2idx = {o:i for i,o in enumerate(users)}
        self.movieid2idx = {o:i for i,o in enumerate(movies)}

        self.idx2userid = {i:o for o,i in self.userid2idx.items()}
        self.idx2movieid = {i:o for o,i in self.movieid2idx.items()}

        self.ratings.movieId = ratings_df.movieId.apply(lambda x: self.movieid2idx[x])
        self.ratings.userId = ratings_df.userId.apply(lambda x: self.userid2idx[x])

        # userid and corresponding movieid that they rated
        self.x = self.ratings.drop(['rating', 'timestamp'], axis=1).values
        # rating
        self.y = self.ratings['rating'].values
        self.x, self.y = torch.tensor(self.x), torch.tensor(self.y)

    def __getitem__(self, index):
        return (self.x[index], self.y[index])

    def __len__(self):
        return len(self.ratings)

In [ ]:
def train_eval_dev(train_df, dev_df, sgd_param):
  
  n_users = ratings_df['userId'].nunique()
  n_items = ratings_df['movieId'].nunique()

  model = MatrixFactorization(n_users, n_items, n_factors=sgd_param['n_factors'])
  optimizer = optim.SGD(model.parameters(), lr=sgd_param['lr'])

  cuda = torch.cuda.is_available()

  if cuda:
    model = model.cuda()

  loss_fn = nn.MSELoss()
  train_set = Loader(train_df)
  # This splits dataset into batches
  train_loader = DataLoader(train_set, batch_size=128, shuffle=True)

  model.train()

  for epoch in range(sgd_param['epochs']):
    total_loss = 0
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.cuda(), target.cuda()
        optimizer.zero_grad()
        # forward pass
        output = model(data)
        # calculate loss
        loss = loss_fn(output, target.float())
        # calculate gradient
        loss.backward()
        # update parameters (embeddings in this case)
        optimizer.step()
        total_loss += loss.item()

  dev_set = Loader(dev_df)
  dev_loader = DataLoader(dev_set, batch_size=128, shuffle=False)
  # model to eval mode
  model.eval()
  dev_preds, dev_targets = [], []
  with torch.no_grad():
      for data, target in dev_loader:
          data, target = data.cuda(), target.cuda()
          output = model(data)
          # append results
          dev_preds.extend(output.cpu().numpy())
          dev_targets.extend(target.cpu().numpy())

  mae = mean_absolute_error(dev_targets, dev_preds)

  print(f"dev_mae for {sgd_param['n_factors']} factors, {sgd_param['lr']} lr, {sgd_param['epochs']} epochs: {mae}")

In [ ]:
for n_f in [10, 20, 50, 100]:
  for n_ep in [10, 50, 100]:
    for lr in [0.001, 0.01, 0.05]:
      train_eval_dev(train_df, dev_df, {'n_factors': n_f, 'epochs': n_ep, 'lr': lr})

# Results

dev_mae for 10 factors, 0.001 lr, 10 epochs: 3.4900412809292685
dev_mae for 10 factors, 0.01 lr, 10 epochs: 3.480439918096354
dev_mae for 10 factors, 0.05 lr, 10 epochs: 3.028467126331669
dev_mae for 10 factors, 0.001 lr, 50 epochs: 3.486756268621376
dev_mae for 10 factors, 0.01 lr, 50 epochs: 3.026092983281317
dev_mae for 10 factors, 0.05 lr, 50 epochs: 1.3760415739293925
dev_mae for 10 factors, 0.001 lr, 100 epochs: 3.480752333602614
dev_mae for 10 factors, 0.01 lr, 100 epochs: 2.1650907983948624
dev_mae for 10 factors, 0.05 lr, 100 epochs: 1.1044064729262975
dev_mae for 20 factors, 0.001 lr, 10 epochs: 3.483121689989763
dev_mae for 20 factors, 0.01 lr, 10 epochs: 3.4644898156541255
dev_mae for 20 factors, 0.05 lr, 10 epochs: 2.8658078690881816
dev_mae for 20 factors, 0.001 lr, 50 epochs: 3.476532301745354
dev_mae for 20 factors, 0.01 lr, 50 epochs: 2.869004440047278
dev_mae for 20 factors, 0.05 lr, 50 epochs: 1.3476828530524387
dev_mae for 20 factors, 0.001 lr, 100 epochs: 3.4646104475111694
dev_mae for 20 factors, 0.01 lr, 100 epochs: 2.0680492953973366
dev_mae for 20 factors, 0.05 lr, 100 epochs: 1.0965616834833602
dev_mae for 50 factors, 0.001 lr, 10 epochs: 3.4622471793977434
dev_mae for 50 factors, 0.01 lr, 10 epochs: 3.4174553854509817
dev_mae for 50 factors, 0.05 lr, 10 epochs: 2.6361976552181994
dev_mae for 50 factors, 0.001 lr, 50 epochs: 3.445969564824434
dev_mae for 50 factors, 0.01 lr, 50 epochs: 2.6354201712233962
dev_mae for 50 factors, 0.05 lr, 50 epochs: 1.3040099250541246
dev_mae for 50 factors, 0.001 lr, 100 epochs: 3.4169696260114346
dev_mae for 50 factors, 0.01 lr, 100 epochs: 1.9297473189003211
dev_mae for 50 factors, 0.05 lr, 100 epochs: 1.0835130303438045
dev_mae for 100 factors, 0.001 lr, 10 epochs: 3.4275661815249725
dev_mae for 100 factors, 0.01 lr, 10 epochs: 3.3410197442210903
dev_mae for 100 factors, 0.05 lr, 10 epochs: 2.438167753674767
dev_mae for 100 factors, 0.001 lr, 50 epochs: 3.396717071175329
dev_mae for 100 factors, 0.01 lr, 50 epochs: 2.440149415787094
dev_mae for 100 factors, 0.05 lr, 50 epochs: 1.2641956796488487
dev_mae for 100 factors, 0.001 lr, 100 epochs: 3.341480087737653
dev_mae for 100 factors, 0.01 lr, 100 epochs: 1.8127504039392
dev_mae for 100 factors, 0.05 lr, 100 epochs: 1.0718658247723067 <- Best

In [ ]:
n_users = ratings_df['userId'].nunique()
n_items = ratings_df['movieId'].nunique()

model = MatrixFactorization(n_users, n_items, n_factors=100)
optimizer = optim.SGD(model.parameters(), lr=0.05)

cuda = torch.cuda.is_available()

if cuda:
    model = model.cuda()

loss_fn = nn.MSELoss()
train_set = Loader(train_df)
train_loader = DataLoader(train_set, batch_size=128, shuffle=True)

model.train()

for epoch in range(100):
    total_loss = 0
    for batch_idx, (data, target) in enumerate(train_loader):
        if cuda:
            data, target = (d.cuda() for d in data), target.cuda()
        optimizer.zero_grad()
        output = model(data)
        loss = loss_fn(output, target.float())
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f'Epoch {epoch+1}, Loss: {total_loss / len(train_loader)}')

test_set = Loader(test_df)
test_loader = DataLoader(test_set, batch_size=128, shuffle=False)

model.eval()
test_preds, test_targets = [], []
with torch.no_grad():
    for data, target in test_loader:
        if cuda:
            data, target = (d.cuda() for d in data), target.cuda()
        output = model(data)
        test_preds.extend(output.cpu().numpy())
        test_targets.extend(target.cpu().numpy())

mae = mean_absolute_error(test_targets, test_preds)
mse = mean_squared_error(test_targets, test_preds)

print(f'Mean Absolute Error: {mae}')
print(f'Mean Squared Error: {mse}')